# SQLALchemy 1.4 / 2.0 Tutorial
- [Official Documentaiton](https://docs.sqlalchemy.org/en/14/tutorial/engine.html)

In [7]:
%load_ext lab_black

In [9]:
import sqlalchemy

assert sqlalchemy.__version__ == "1.4.31"

In [10]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [11]:
from sqlalchemy import Table, Column, Integer, String

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [12]:
from sqlalchemy import insert

stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")

In [13]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [14]:
compiled = stmt.compile()

In [16]:
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

In [18]:
print(compiled)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [20]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
metadata_obj.create_all(engine)

2022-02-27 15:23:45,359 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-27 15:23:45,363 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-02-27 15:23:45,366 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-27 15:23:45,369 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-02-27 15:23:45,372 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-27 15:23:45,378 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-02-27 15:23:45,382 INFO sqlalchemy.engine.Engine [no key 0.00354s] ()
2022-02-27 15:23:45,385 INFO sqlalchemy.engine.Engine COMMIT


In [21]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2022-02-27 15:23:47,800 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-27 15:23:47,802 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-02-27 15:23:47,805 INFO sqlalchemy.engine.Engine [generated in 0.00520s] ('spongebob', 'Spongebob Squarepants')
2022-02-27 15:23:47,810 INFO sqlalchemy.engine.Engine COMMIT


In [22]:
result.inserted_primary_key

(1,)

In [23]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

2022-02-27 15:25:20,217 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-27 15:25:20,221 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-02-27 15:25:20,227 INFO sqlalchemy.engine.Engine [generated in 0.01046s] (('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'))
2022-02-27 15:25:20,234 INFO sqlalchemy.engine.Engine COMMIT


In [25]:
from sqlalchemy import ForeignKey

address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

In [29]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

from sqlalchemy.orm import relationship


class User(Base):
    __tablename__ = "user_account"
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"


class Address(Base):
    __tablename__ = "address"
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [30]:
# the identical MetaData object is also present on the
# declarative base
Base.metadata.create_all(engine)

2022-02-27 15:28:50,006 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-27 15:28:50,023 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-02-27 15:28:50,030 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-27 15:28:50,083 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-02-27 15:28:50,085 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-27 15:28:50,088 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-02-27 15:28:50,090 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-27 15:28:50,096 INFO sqlalchemy.engine.Engine 
CREATE TABLE address (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES user_account (id)
)


2022-02-27 15:28:50,111 INFO sqlalchemy.engine.Engine [no key 0.01865s] ()
2022-02-27 15:28:50,116 INFO sqlalchemy.engine.Engine COMMIT


In [31]:
from sqlalchemy import select, bindparam

scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {"username": "spongebob", "email_address": "spongebob@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()

2022-02-27 15:28:51,848 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-27 15:28:51,853 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2022-02-27 15:28:51,855 INFO sqlalchemy.engine.Engine [cached since 75.94s ago] (('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'))
2022-02-27 15:28:51,866 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt)

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


In [33]:
insert_stmt = insert(address_table).returning(
    address_table.c.id, address_table.c.email_address
)
print(insert_stmt)

INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


In [34]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


In [35]:
from sqlalchemy import select

stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [36]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-02-27 15:32:30,853 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-27 15:32:30,857 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-02-27 15:32:30,864 INFO sqlalchemy.engine.Engine [generated in 0.01096s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2022-02-27 15:32:30,870 INFO sqlalchemy.engine.Engine ROLLBACK


In [38]:
from sqlalchemy.orm import Session

stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-02-27 15:33:23,603 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-27 15:33:23,657 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-02-27 15:33:23,673 INFO sqlalchemy.engine.Engine [generated in 0.01680s] ('spongebob',)
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2022-02-27 15:33:23,680 INFO sqlalchemy.engine.Engine ROLLBACK


In [39]:
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [40]:
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [41]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
